In [ ]:
# default_exp pipeline

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
# export
from counternet.import_essentials import *
from counternet.utils.all import *
from counternet.training_module import BaseModule
from counternet.model import BaselinePredictiveModel, CounterNetModel

Global seed set to 31


## Utils

In [ ]:
# export
def load_trained_model(module: BaseModule, checkpoint_path: str, gpus : int = 0) -> BaseModule:
    # assuming checkpoint_path = f"{dict_path}/epoch={n_epoch}-step={step}.ckpt"
    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f'{checkpoint_path} is not found.')

    n_iter = int(checkpoint_path.split("-")[0].split("=")[-1]) + 1
    model = module.load_from_checkpoint(checkpoint_path)
    tmp_trainer = pl.Trainer(
        max_epochs=n_iter, resume_from_checkpoint=checkpoint_path, num_sanity_val_steps=0, gpus=gpus,
        logger=False, checkpoint_callback=False
    )
    tmp_trainer.fit(model)

    return model

## Trainer

In [ ]:
# export
class ModelTrainer(object):
    def __init__(self,
                 model: BaseModule,
                 t_configs: Dict[str, Any],
                 callbacks: Optional[List[Callback]] = None,
                 description: Optional[str] = None,
                 debug: Optional[bool] = False,
                 logger: Optional[LightningLoggerBase] = None,
                 logger_name: str = "debug"):

        if logger is None:
            logger = pl_loggers.TestTubeLogger(
                Path('../log/'),
                name=logger_name,
                description=description,
                debug=debug,
                log_graph=True
            )

        # model checkpoint
        checkpoint_callback = ModelCheckpoint(
            monitor='val/val_loss',
            save_top_k=3,
            mode='min'
        )

        # define callbacks
        if callbacks is None:
            callbacks = [checkpoint_callback]
        elif self._has_no_model_checkpoint(callbacks):
            callbacks += [checkpoint_callback]

        self.trainer = pl.Trainer(
            logger=logger,
            callbacks=callbacks,
            **t_configs
        )

        self.model = model

    def _has_no_model_checkpoint(self, callbacks: List[Callback]) -> bool:
        for callback in callbacks:
            if isinstance(callback, ModelCheckpoint):
                return False
        return True

    def fit(self, is_parallel=False):
        if is_parallel:
            logging.warning(
                f"parallel version has not been implemented\nUsing the single process training...")
        self.trainer.fit(self.model)

        return dict(
            trainer=self.trainer,
            module=self.model
        )

    def load_trained_model(self, checkpoint_path: str, gpus: int = 0) -> BaseModule:
        self.model = load_trained_model(
            self.model, checkpoint_path=checkpoint_path, gpus=gpus)
        return self.model

In [ ]:
t_config = load_json('assets/configs/trainer.json')
m_config = load_json('assets/configs/adult.json')
model = BaselinePredictiveModel(m_config)
model_trainer = ModelTrainer(
    model, t_config
)
model_trainer.fit()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name        | Type              | Params | In sizes | Out sizes
-------------------------------------------------------------------------
0 | val_acc     | Accuracy          | 0      | ?        | ?        
1 | model       | Sequential        | 2.1 K  | [1, 29]  | [1, 1]   
2 | sensitivity | SensitivityMetric | 0      | ?        | ?        
-------------------------------------------------------------------------
2.1 K     Trainable params
0         Non-trainable params
2.1 K     Total params
0.009     Total estimated model params size (MB)
train (tensor([[0.4184, 0.4490, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [0.3163, 0.3980, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [0.3469, 0.3980, 0.0000,  ..., 1.0000, 1.0000, 0.0000],
        ...,
        [0.3878, 0.3980, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.3571, 0.3469, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [0.1939, 0.4796, 0.0000,  

/home/birk/software/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/birk/software/miniconda3/lib/python3.8/site-packages/pytorch_lightning/core/step_result.py:115: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = torch.tensor(value, device=device, dtype=torch.float)
Global seed set to 31
/home/birk/software/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Conside

{'trainer': <pytorch_lightning.trainer.trainer.Trainer at 0x7f45f405a5e0>,
 'module': BaselinePredictiveModel(
   (val_acc): Accuracy()
   (model): Sequential(
     (0): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (block): Sequential(
             (0): Linear(in_features=29, out_features=50, bias=True)
             (1): LeakyReLU(negative_slope=0.01)
             (2): Dropout(p=0.3, inplace=False)
           )
         )
         (1): LinearBlock(
           (block): Sequential(
             (0): Linear(in_features=50, out_features=10, bias=True)
             (1): LeakyReLU(negative_slope=0.01)
             (2): Dropout(p=0.3, inplace=False)
           )
         )
       )
     )
     (1): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (block): Sequential(
             (0): Linear(in_features=10, out_features=10, bias=True)
             (1): LeakyReLU(negative_slope=0.01)
             (2): Dropout(p=0

## CF Generator
- local explainer
- global explainer